In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import copy
import scipy.stats as sps
from sympy import *
from matplotlib import cm
plt.style.use('ggplot')
%matplotlib inline

def scan_from_csv(filename,header=None):
    if not header:
        return pd.read_csv(filename,na_values='None')
    else:
        return pd.read_csv(filename,na_values='None',header=-1)

def write_answer_to_file(answer,file):
    with open(file, 'w') as answer_file:  
        answer_file.write(answer)

In [135]:
dataF = pd.read_csv('flare.data1.txt',delimiter='\\s+',header=-1)

In [136]:
dataF.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,C,S,O,1,2,1,1,2,1,2,0,0,0
1,D,S,O,1,3,1,1,2,1,2,0,0,0
2,C,S,O,1,3,1,1,2,1,1,0,0,0
3,D,S,O,1,3,1,1,2,1,2,0,0,0
4,D,A,O,1,3,1,1,2,1,2,0,0,0


оставим только 10 первых столбцов(преобразуем символы в их числовое представление)

In [137]:
data = copy.copy(dataF)
for i in range(3):
    data[i] = [ord(x) for x in list(data[i])]
data = data[list(np.arange(10))]
data = np.array(data)
print(data)

[[67 83 79 ...,  2  1  2]
 [68 83 79 ...,  2  1  2]
 [67 83 79 ...,  2  1  1]
 ..., 
 [69 65 79 ...,  2  1  2]
 [67 82 79 ...,  2  1  1]
 [69 75 67 ...,  2  2  2]]


In [138]:
len(data)

323

Проверим сложную гипотезу о независимости столбцов в совокупности с помощью таблиц сопряженности, причем применим этот метод для каждой пары столбцов на  разбиениях $\mathbb{R}$ .

Далее при составлении таблиц сопряженности неизбежно будем получать иногда нулевые ячейки, так как данных не очеь много. Поэтому закинем в каждую ячейку по одному элементу(иначе критерий просто не сработает)

Вообще здесь применение этого теста не очень оправдано.
По хорошему тут нужно применять точный критерий Фишера, но кажется его аналог не реализован не для бинарных разбиений.
Поэтому посчитаем и его для всех бинарных разбиений.

Для проверки сложной гипотезы будем использовать нисходящий метод Холма.

Рассмотрим проверку гипотезы на уровне $\alpha = 0.05$ 

Всего простых гипотез: $ 2 \cdot C_{10}^2 = 90$

In [182]:
from statsmodels.sandbox.stats.multicomp import multipletests  # Holm method
from scipy.stats import chi2_contingency  # Contigency Table statistic
from scipy.stats import fisher_exact  # exact Fisher crit.

In [192]:
def make_contigency_table(X,Y,method='auto'):
    junk, binx = np.histogram(X,bins=method)
    junk, biny = np.histogram(Y,bins=method)
    
    table = np.ones((len(binx)-1,len(biny)-1))
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            table[i][j] = np.sum([(x > binx[i] and x <= binx[i+1] and y > biny[j] and y <= biny[j+1]) 
                                  for x,y in zip(X,Y)])
    for i in range(len(binx)-1):
        for j in range(len(biny)-1):
            if table[i][j] == 0:
                table[i][j] = 1
    return table

In [193]:
# adaptive criteries
run_criteries = [
    (lambda x, y: chi2_contingency(make_contigency_table(x,y))[1]),
    (lambda x, y: fisher_exact(make_contigency_table(x, y, 2))[1])
]

In [194]:
alphas = [0.05]
pairs = []
for a in alphas:
    p_values = []
    for i in range(10):
        for j in range(i+1,10):
            for ex in run_criteries:
                pairs.append((i,j))
                p_values.append(ex(data[:,i],data[:,j]))
    reject = multipletests(pvals=p_values, method='holm',alpha=a)[0]
    print('for alpha = ' + str(a) + ' : ' + str(np.sum(reject)) + ' simple hypotheses are rejected:')
    print('p_values: ' + str(p_values))
    print('reject_bits: ' + str(reject))
    print()    
print('pairs', str(list(zip(pairs,reject))))

for alpha = 0.05 : 7 simple hypotheses are rejected:
p_values: [0.00017169595685431841, 0.0043016029238558311, 3.5258010143870008e-33, 1.6268996069967976e-08, 1.0, 0.31585365853658626, 0.0012265057738787026, 0.045654946780133182, 1.0, 0.36551724137931002, 0.7873190735405815, 0.39068508759436055, 1.0, 0.41485562697683415, 1.0, 0.34545454545454563, 1.0, 0.53091243392327747, 2.2666796018750134e-10, 1.0760568960322799e-09, 1.0, 1.0, 1.964706713499357e-06, 0.29203255687304736, 1.0, 1.0, 0.36304003006099744, 1.0, 1.0, 0.42460163233575088, 1.0, 0.31818181818181812, 1.0, 0.51401614310647892, 1.0, 1.0, 3.6967804483255583e-10, 0.0044393107039925909, 1.0, 1.0, 0.42714969074286835, 1.0, 1.0, 0.52660034602080319, 1.0, 1.0, 1.0, 0.52867334606838245, 1.0, 1.0, 1.0, 0.15666666666666676, 1.0, 0.1229838709677418, 1.0, 0.08244680851063832, 1.0, 0.37777777777777766, 1.0, 0.096341463414634065, 1.0, 1.0, 0.0069454984185440584, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.15666666666666676, 1.0, 0.1158645276292

В итоге, сложная гипотеза о независимости в совокупности отвергается на уровне 0.05, так как присутствуют пары столбцов, для которых гипотеза о независимости отвергается, а именно, для пар:

для хи2-критерия:
(0,1),(0,2),(1,2),(1,4),(2,4)  

для точного критерия Фишера:
(0,2),(1,2)

В принципе критерии дали похожие результаты.

Посмотрим, что это были за признаки:

1. Code for class (modified Zurich class)  (A,B,C,D,E,F,H)
2. Code for largest spot size              (X,R,S,A,H,K)
3. Code for spot distribution              (X,O,I,C)
4. Activity                                (1 = reduced, 2 = unchanged)
5. Evolution                               (1 = decay, 2 = no growth, 3 = growth)
6. Previous 24 hour flare activity code    (1 = nothing as big as an M1,2 = one M1, 3 = more activity than one M1)
7. Historically-complex                    (1 = Yes, 2 = No)
8. Did region become historically complex  (1 = yes, 2 = no) 
  on this pass across the sun's disk
9. Area                                    (1 = small, 2 = large)
10. Area of the largest spot                (1 = <=5, 2 = >5)

<тут должна быть фраза объясняющая зависимости между этими признаками>
видимо независимость отверглась в связях между признаками являющимися свойствами solar flare а не свойствами его местоположения(которые кажется более случайны, чем характеристики самого объекта)